# Проект 13 создание модели для поиска токсичных комментариев

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
#import pip
#pip.

In [2]:
import pandas as pd
import numpy as np

#import torch
#import transformers
#from tqdm import notebook

#from nltk.corpus import stopwords
from nltk.corpus import stopwords as nltk_stopwords
import nltk

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import re
from pymystem3 import Mystem
m = Mystem()

from nltk.stem import SnowballStemmer 
english_stemmer = SnowballStemmer('english')

In [3]:
#data = pd.read_csv('toxic_comments.csv')
data = pd.read_csv('/datasets/toxic_comments.csv')

#временная мера для уменьшения данных:
#data = data.sample(10000).reset_index(drop=True)

#Приведём данные в более единый формат, методом приведения к нижнему регистру:
data['text'] = data['text'].str.lower()

#data

In [4]:
def add_lemmatize_and_clear_text(data,column_name):
    lemmatize_and_clear_text_data = []
    for index in data.index:
        row_text = data.loc[index,column_name]
    
        re_sub = re.sub(r'[^a-zA-Z ]', ' ', row_text)
        re_sub_split = re_sub.split()
        re_sub_split_text = " ".join(re_sub_split)
        
        lemm_list = english_stemmer.stem(re_sub_split_text)
        #lemm_list = m.lemmatize(re_sub_split_text) #pymystem3
        lemm_text = "".join(lemm_list)
        
        lemmatize_and_clear_text_data.append(lemm_text)
    return lemmatize_and_clear_text_data
#
data['lemm_text'] = add_lemmatize_and_clear_text(data,'text')

In [5]:
#выборочно сократим датасет, чтобы уровнять пропорции токсичных и нетоксичных сообщений

data_toxic = data.loc[data['toxic']==1]

data_nan_toxic = data.loc[data['toxic']==0]
data_nan_toxic = data_nan_toxic.sample(len(data_toxic)).reset_index(drop=True)

data_proportional = pd.concat([data_toxic, data_nan_toxic])
data_proportional = data_proportional.sample(frac=1).reset_index(drop=True) 
#data

In [6]:
data_proportional

,text,toxic,lemm_text
0,go f yourself\n\nyou are so typical of your ty...,1,go f yourself you are so typical of your type ...
1,hino contessa \n\nthe article refers to the co...,0,hino contessa the article refers to the contes...
2,the basis would be the 18 ju 86s operated by t...,0,the basis would be the ju s operated by the so...
3,dare send messages like this. you are the caus...,1,dare send messages like this you are the cause...
4,"""\nback to bragging i see. you don't even hav...",1,back to bragging i see you don t even have a c...
...,...,...,...
32445,"""\n\ncomley stop accusing me of being a vandly...",1,comley stop accusing me of being a vandly pak ...
32446,"well, better. i will help improve your userpag...",0,well better i will help improve your userpages...
32447,you're an 7åss fùck8 \n\nyou're an 7åss fùck8\...,1,you re an ss f ck you re an ss f ck go sh t yo...
32448,"""so besides being brainwashed by a cult what i...",0,so besides being brainwashed by a cult what is...


## Комментарий наставника
<span style="color:orange">Хорошо, что борешься с дисбалансом в датасете, но для задач обработки естественного языка требуется как можно больший объем данных для успешного обучения модели, поэтому будь осторожен при избавлении от такого количества ценных данных. Из 159571 записи осталось только 32450.</span>

In [7]:
df_train, df_valid_and_test = train_test_split(data_proportional, test_size=0.4, random_state=12345)
df_test, df_valid = train_test_split(df_valid_and_test, test_size=0.5, random_state=12345)

train_corpus = df_train['lemm_text'].values.astype('U')
train_target = df_train['toxic']

valid_corpus = df_valid['lemm_text'].values.astype('U')
valid_target = df_valid['toxic']

test_corpus = df_test['lemm_text'].values.astype('U')
test_target = df_test['toxic']

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
count_tf_idf.fit(train_corpus)

train_tf_idf = count_tf_idf.transform(train_corpus)

valid_tf_idf = count_tf_idf.transform(valid_corpus)

test_tf_idf = count_tf_idf.transform(test_corpus)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Комментарий наставника
<span style="color:green">Векторизация проведена абсолютно верно.</span> \
<span style="color:orange">Для задач NLP можешь оставлять намного меньше данных под валидацию и тест (можно даже по 5% на каждую выборку). Тренировочная выборка важнее.</span>

# 2. Обучение

In [8]:
%%time
# LogisticRegression
from sklearn.linear_model import LogisticRegression

ModelLogisticRegression = LogisticRegression(random_state=0)
ModelLogisticRegression.fit(train_tf_idf, train_target)

from sklearn.metrics import f1_score

# на валидационных данных F1 = :
predictions = ModelLogisticRegression.predict(valid_tf_idf)
print(f1_score(predictions, valid_target))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.884493670886076
CPU times: user 2.77 s, sys: 2.05 s, total: 4.82 s
Wall time: 4.84 s


## Комментарий наставника
<span style="color:orange">Будь осторожен: по документации sklearn первым параметром в функции метрик всегда подаются реальные (целевые) значения, а только потом уже предсказания. Для F1 последовательность роли не играет, но для других метрик она может быть важна.</span>

In [9]:
# на тестовых данных F1 = :
predictions = ModelLogisticRegression.predict(test_tf_idf)
print(f1_score(predictions, test_target))

0.8814153649984061


In [10]:
%%time
from sklearn.ensemble import RandomForestClassifier

ModelRandomForestClassifiern = RandomForestClassifier(random_state=0)
ModelRandomForestClassifiern.fit(train_tf_idf, train_target)

# на валидационных данных F1 =:
predictions = ModelRandomForestClassifiern.predict(valid_tf_idf)
print(f1_score(predictions, valid_target))

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8068484042553193
CPU times: user 6.14 s, sys: 15.2 ms, total: 6.16 s
Wall time: 6.3 s


In [11]:
# на тестовых данных F1 = :
predictions = ModelRandomForestClassifiern.predict(test_tf_idf)
print(f1_score(predictions, test_target))

0.8069651741293531


# 3. Выводы

Выборочное сокращение датасета - дало отличный результат. Модель меньше училось, но более точна.

скорость LogisticRegression намного быстрее чем у RandomForestClassifier

Хотя по метрике качества - она её превосходит

# Закомментированный BERT

In [12]:
#tokenizer = transformers.BertTokenizer(vocab_file='/datasets/ds_bert/vocab.txt')

#tokenized = data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

#max_len = 0
#for i in tokenized.values:
#    if len(i) > max_len:
#        max_len = len(i)

#padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

#attention_mask = np.where(padded != 0, 1, 0)

#config = transformers.BertConfig.from_json_file('/datasets/ds_bert/bert_config.json')
#model = transformers.BertModel.from_pretrained('/datasets/ds_bert/rubert_model.bin', config=config)

In [13]:
#batch_size = 100
#embeddings = []
#for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
#        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
#        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
#        
#        with torch.no_grad():
#            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
#        
#        embeddings.append(batch_embeddings[0][:,0,:].numpy())
#

In [14]:
#features = np.concatenate(embeddings)
#target = data['toxic']